# IDOM - A Brief Introduction

IDOM is an easily installable and powerful Python package for:

- Making responsive user interfaces with pure Python

- Doing so with a declarative decorator-based API

- And rendering those layouts on the web or in Jupyter Notebooks


# This Notebook

Will guide you though the basics of IDOM through several examples:

1. [Slideshows - Basic Interactivity](#Slideshows---Basic-Interactivity)

2. [To Do Lists - More on Interactivity](#To-Do-Lists---More-on-Interactivity)

3. [Plotting with Matplotlib - A Simple Dashboard](#Plotting-with-Matplotlib---A-Simple-Dashboard)

4. [Drag and Drop - Complex Interface Features](#Drag-and-Drop---Complex-Interface-Features)

5. [The Game Snake - Using What We've Learned](#The-Game-Snake---Using-What-We've-Learned)

6. [Importing Javascript - Leveraging The Ecosystem](#Importing-Javascript---Leveraging-The-Ecosystem)

7. [Sharing Views - A Common Model State](#Sharing-Views---A-Common-Model-State)


## Where Can You Learn More?

Go to https://github.com/rmorshea/idom for:

- more documentation

- installation instructions

- trouble shooting support


In [ ]:
import idom

### A Little Set Up

Since you may be running this notebook on [mybinder.org](https://mybinder.org) we have to do a little bit of work to make sure that your frontend view connects to the right URL. You shouldn't have to change anything here. Be sure to [post an issue](https://github.com/rmorshea/idom/issues) if you experience any problems. 

In [ ]:
from idom.server.sanic import PerClientState
from example_utils import example_uri_root, html_link, pretty_dict_string

websocket_url = example_uri_root("ws", 8765) + "/stream"
webpage_url = html_link(example_uri_root("http", 8765) + "/client/index.html")

mount, element = idom.hotswap()
PerClientState(element).daemon("localhost", 8765, access_log=False)

def display(*args, **kwargs):
    element, *args = args
    mount(element, *args, **kwargs)
    return idom.display("jupyter", websocket_url)

# Slideshows - Basic Interactivity

Let's use IDOM to create a simple slideshow which changes whenever a user clicks an image. To do this we define a function called `Slideshow` which will return an `<img/>` element. Attached to this `<img/>` is a handler for `onClick` events created whenever a user clicks the image.

In [ ]:
@idom.element
async def Slideshow(self, index=0):

    async def update_image(event):
        self.update(index + 1)

    url = f"https://picsum.photos/800/300?image={index}"
    return idom.html.img({"src": url, "onClick": update_image})

## Try it Out!

The following cells will display the output here:

In [ ]:
print("Try clicking the image! 🖱️")

display(Slideshow)

## It Works Outside the Notebook Too!

However you can also see the same thing in a standard web page:

In [ ]:
webpage_url

## What Just Happened?

At the core of IDOM are simple dictionary structures that model an HTML layout.

To help you understand what's happening let's see what this dictionary model looks like:

In [ ]:
slideshow = idom.Layout(Slideshow())
_, changes, _ = await slideshow.render()
print(pretty_dict_string(changes))

This will render the following HTML

```html
<img src='https://picsum.photos/800/300?image={index}' onClick={slideshow-change-callback}/>
```

Where each time the user clicks the rendered image:

1. The `slideshow-change-callback` is triggered and calls `change()`

2. The `change()` function then tells the `Slideshow` element to update with a new `{index}`.

3. A new image is rendered for the user with the updated `src` index.

# To Do Lists - More on Interactivity

So far we've created event handlers which respond to simpler events like clicks, but often we want to track more complicated events. In this example we'll create a to-do list. To accomplish this goal we'll need to accept user inputs and track key presses by using the `event` dictionary which is passed to event handlers. The simple example below shows how to create a simple `<input/>` which tracks the `onChange` in order to print out its current value from `event["value"]`:

```python
@idom.element
async def SimpleInput(self):

    async def change(event):
        print(event["value"])

    idom.html.input(onChange=change)
```

There's lots more information that you can get from the `event` object though. For a full list you can refer to [React's documentation](https://reactjs.org/docs/events.html#supported-events). For example the [keyboard events](https://reactjs.org/docs/events.html#keyboard-events) has a `key` attribute containing the name of the current key being pressed (e.g. "ArrowLeft", "h", or "2"). All event attributes can by found within the `event` dictionary.

## The Code

In [ ]:
@idom.element
async def Todo(self):
    items = []

    async def add_new_task(event):
        if event["key"] == "Enter":
            items.append(event["value"])
            task_list.update(items)

    task_input = idom.html.input({"onKeyDown": add_new_task})
    task_list = TaskList(items)

    return idom.html.div([task_input, task_list])


@idom.element
async def TaskList(self, items):
    tasks = []

    for index, text in enumerate(items):

        async def remove(event, index=index):
            del items[index]
            self.update(items)

        task_text = idom.html.td([idom.html.p([text])])
        delete_button = idom.html.td({"onClick": remove}, [idom.html.button(["x"])])
        tasks.append(idom.html.tr([task_text, delete_button]))

    return idom.html.table(tasks)


print("Try typing in the text box and pressing 'Enter' 📋")

display(Todo)

# Plotting with Matplotlib - A Simple Dashboard

The following example shows how you might create an animated matplotlib plot of 1D [Brownian Motion](https://en.wikipedia.org/wiki/Brownian_motion) over time. Matplotlib provides similar functionality natively, though with the limitation that all the frames of the animation are pre-computed. With `idom` we can extend the animation out indefinitely, and even modify it on the fly based on user input.

At the moment `idom` doesn't have native support for Matplotlib, but you can still render them by:

1. Creating an `Image` element with some format (e.g. SVG).

2. Calling a figure's `savefig` method and writing to the `Image.io` attribute.

In [ ]:
%matplotlib agg
from matplotlib import pyplot as plt

## Animation Callback

The animation callback is a special hook registered by decorating a function with `self.animate` upon rendering an element. This hook is called repeatedly each time a render takes place allowing you to update a view without having to wait for user interaction. In the case of our plot we use it to create a new frame of the Brownian Particle's path.


## Input Helpers

In the last example we saw how you can create an `<input>` element and register callbacks to it. Here though, we use `Input` object which allows you to capture the current value inside an input by accessing the `Input.value` attribute. You're still able to register event handlers via the `Input.events` attribute which an `Events` object. 

In this case we use the `Input.value` attribute to recompute the graph. We also use the events object to synchronize the values between two different inputs. For example we want the value on slider and text inputs
for the `mu` and `sigma` inputs to be the same.

In [ ]:
import time
import asyncio
import random


@idom.element
async def RandomWalk(self):
    x, y = [0] * 50, [0] * 50
    plot = Plot(x, y)

    mu_var, mu_inputs = linked_inputs(
        "Mean", 0, "number", "range", min=-1, max=1, step=0.01
    )
    sigma_var, sigma_inputs = linked_inputs(
        "Standard Deviation", 1, "number", "range", min=0, max=2, step=0.01
    )

    @self.animate(rate=0.3)
    async def walk(stop):
        x.pop(0)
        x.append(x[-1] + 1)
        y.pop(0)
        diff = random.gauss(float(mu_var.get()), float(sigma_var.get()))
        y.append(y[-1] + diff)
        plot.update(x, y)
    
    style = idom.html.style(["""
    .linked-inputs {margin-bottom: 20px}
    .linked-inputs input {width: 48%;float: left}
    .linked-inputs input + input {margin-left: 4%}
    """])

    return idom.html.div({"style": {"width": "60%"}}, [style, plot, mu_inputs, sigma_inputs])


@idom.element(run_in_executor=True)
async def Plot(self, x, y):
    fig, axes = plt.subplots()
    axes.plot(x, y)
    img = idom.Image("svg")
    fig.savefig(img.io, format="svg")
    plt.close(fig)
    return img


def linked_inputs(label, value, *types, **attributes):
    var = idom.Var(value)

    inputs = []
    for t in types:
        inp = idom.Input(t, value, **attributes)

        @inp.events.on("change")
        async def on_change(inp, event):
            for i in inputs:
                i.update(inp.value)
            var.set(inp.value)

        inputs.append(inp)

    fs = idom.html.fieldset({"class": "linked-inputs"}, [idom.html.legend(label)], inputs)

    return var, fs


print("Try clicking the plot! 📈")

display(RandomWalk)

# Drag and Drop - Complex Interface Features

The examples we've seen are already a strong testimate to the power of IDOM, but there's even more that you can do. IDOM is capable of creating drag and drop tools. This is great because dragging and dropping is one of the most satisfying experiences as a user because it's incredibly intuitive, and is effortlessly smooth.

## Advanced Event Options

In the above examples, event handlers have been assigned to element attributes directly. However in order to implement drag and drop we need to `stopPropagation` of an event, and `preventDefault` actions the event takes. To enable you to execute on this there is an `idom.event` decorator that can be used to indicate these more advanced options:

```python
@idom.event(preventDefault=True, stopPropagation=True)
async def my_callback(event):
    ...
```


## Useful Tools

While writing code that involves side effects is not usually a good idea, there are some situations where they are useful. In the drag-and-drop example below we need to keep track of which box is filled and which is not. To do this we use an `idom.Var` a simple utility that holds a reference to an object. In this case we use it to holds a reference to the `Holder` which is currently filled. Once a new holder should become filled we use the `Var` to sneakily update make the last holder empty.

In [ ]:
@idom.element
async def DragDropBoxes(self):
    last_owner =idom.Var(None)
    last_hover = idom.Var(None)

    h1 = Holder("filled", last_owner, last_hover)
    h2 = Holder("empty", last_owner, last_hover)
    h3 = Holder("empty", last_owner, last_hover)

    last_owner.set(h1)

    style = idom.html.style(["""
    .holder {
      height: 150px;
      width: 150px;
      margin: 20px;
      display: inline-block;
    }
    .holder-filled {
      border: solid 10px black;
      background-color: black;
    }
    .holder-hover {
      border: dotted 5px black;
    }
    .holder-empty {
      border: solid 5px black;
      background-color: white;
    }
    """])

    return idom.html.div([style, h1, h2, h3])


@idom.element(state="last_owner, last_hover")
async def Holder(self, kind, last_owner, last_hover):

    @idom.event(preventDefault=True, stopPropagation=True)
    async def hover(event):
        if kind != "hover":
            self.update("hover")
            old = last_hover.set(self)
            if old is not None and old is not self:
                old.update("empty")
    
    async def start(event):
        last_hover.set(self)
        self.update("hover")
    
    async def end(event):
        last_owner.get().update("filled")

    async def leave(event):
        self.update("empty")

    async def dropped(event):
        if last_owner.get() is not self:
            old = last_owner.set(self)
            old.update("empty")
        self.update("filled")

    return idom.html.div({
        "draggable": (kind == "filled"),
        "onDragStart": start,
        "onDragOver": hover,
        "onDragEnd": end,
        "onDragLeave": leave,
        "onDrop": dropped,
        "class": f"holder-{kind} holder",
    })


print("Click and drag the black box onto the white one! 👆")

display(DragDropBoxes)

# The Game Snake - Using What We've Learned

To show off what's possible with IDOM, let's look at how to make the game Snake.


## Adding Persistent Element State

What's new here is the declaration of `state` when creating an `@element` function:

```python
@idom.element(state="x, y")
def MyElement(self, x, y):
    ...
```

Each parameter of `MyElement` specified in `state` will have its value preserved across updates unless explicitely changed. So for example if we instantiated `MyElement(x=1, y=1)` and later called `self.update()` both `x` and `y` would be passed in as `1` again. However calling `self.update(y=2)` would cause `MyElement` to render with `x=1, y=2` instead. Calling `self.update(x=3)` once would have `MyElement` render with `x=3, y=2`.

In our `Game` element we use this to preserve the original `grid_size` and `block_size` after the player loses and we need to reset the game.

In [ ]:
import enum
import time
import random
import asyncio


class Directions(enum.Enum):
    ArrowUp = (-1, 0)
    ArrowLeft = (0, -1)
    ArrowDown = (1, 0)
    ArrowRight = (0, 1)


class GameState:
    
    def __init__(self, grid_size, block_size):
        self.snake = []
        self.grid = Grid(grid_size, block_size)
        self.new_direction = idom.Var(Directions.ArrowRight)
        self.old_direction = idom.Var(Directions.ArrowRight)
        self.food = idom.Var(None)
        self.won = idom.Var(False)
        self.lost = idom.Var(False)


@idom.element(state="grid_size, block_size")
async def GameView(self, grid_size, block_size):
    game = GameState(grid_size, block_size)
    
    grid_events = game.grid["eventHandlers"]

    @grid_events.on("KeyDown", preventDefault=True)
    async def direction_change(event):
        if hasattr(Directions, event["key"]):
            game.new_direction.set(Directions[event["key"]])

    game.snake.extend(
        [
            (grid_size // 2 - 1, grid_size // 2 - 3),
            (grid_size // 2 - 1, grid_size // 2 - 2),
            (grid_size // 2 - 1, grid_size // 2 - 1),
        ]
    )

    grid_points = set((x, y) for x in range(grid_size) for y in range(grid_size))

    def set_new_food():
        points_not_in_snake = grid_points.difference(game.snake)
        new_food = random.choice(list(points_not_in_snake))
        get_grid_block(game.grid, new_food).update("blue")
        game.food.set(new_food)

    @self.animate(rate=0.5)
    async def loop(stop):
        if game.won.get() or game.lost.get():
            await asyncio.sleep(1)
            self.update()
        else:
            await draw(game, grid_size, set_new_food)

    set_new_food()
    return game.grid


async def draw(game, grid_size, set_new_food):
    if game.snake[-1] in game.snake[:-1]:
        # point out where you touched
        get_grid_block(game.grid, game.snake[-1]).update("red")
        game.lost.set(True)
        return

    vector_sum = tuple(
        map(sum, zip(game.old_direction.get().value, game.new_direction.get().value))
    )
    if vector_sum != (0, 0):
        game.old_direction.set(game.new_direction.get())

    new_head = (
        # grid wraps due to mod op here
        (game.snake[-1][0] + game.old_direction.get().value[0]) % grid_size,
        (game.snake[-1][1] + game.old_direction.get().value[1]) % grid_size,
    )

    game.snake.append(new_head)

    if new_head == game.food.get():
        if len(game.snake) == grid_size * grid_size:
            get_grid_block(game.grid, new_head).update("yellow")
            game.won.set(True)
            return
        set_new_food()
    else:
        get_grid_block(game.grid, game.snake.pop(0)).update("white")

    # update head after tail - new head may be the same as the old tail
    get_grid_block(game.grid, new_head).update("black")


def Grid(grid_size, block_size):
    return idom.html.div(
        {
            "style": {
                "height": f"{block_size * grid_size}px",
                "width": f"{block_size * grid_size}px",
            },
            
            "tabIndex": -1,
        },
        [
            idom.html.div(
                {"style": {"height": block_size}},
                [Block("white", block_size) for i in range(grid_size)]
            )
            for i in range(grid_size)
        ],
        event_handlers=idom.Events(),
        
    )


@idom.element(state="block_size")
async def Block(self, color, block_size):
    return idom.html.div(
        {
            "style": {
                "height": f"{block_size}px",
                "width": f"{block_size}px",
                "backgroundColor": color,
                "display": "inline-block",
                "border": "1px solid white",
            }
        }
    )


def get_grid_block(grid, point):
    x, y = point
    return grid["children"][x]["children"][y]


print("Click to start playing and use the arrow keys to move 🎮")
print()
print("Slow internet may cause inconsistent frame pacing 😅")

display(GameView, 8, 50)

# Importing Javascript - Leveraging The Ecosystem

With IDOM, you're able to leverage existing Javascript libraries by specifying packages that you'd like to import and the css needed to style the components. In the following example we display a [date picker](https://ant.design/components/date-picker/) from the [Ant Design System](https://ant.design/) and register a callback to it. The cool part here is that the `onChange` callback conforms to the [specification](https://ant.design/components/date-picker/#DatePicker) defined by Ant and handles two arguments instead of one containing a DOM event dictionary.

Since we've seen what's possible with IDOM all we'll do is print out the event information.

In [ ]:
antd = idom.Import("antd", fallback="Loading...")


@idom.element
async def AntDatePicker(self):

    async def changed(moment, datestring):
        print("CLIENT DATETIME:", moment)
        print("PICKED DATETIME:", datestring)

    return idom.html.div(
        [
            idom.html.link({
                "rel": "stylesheet", "type": "text/css", "href": "https://dev.jspm.io/antd/dist/antd.css"
            }),
            antd.DatePicker({"onChange": changed}),
        ]
    )


print("Select a date to see the event print-out 📅")

display(AntDatePicker)

## More Ant Design

Here's a bunch of other fun components that we can use from Ant Design.

In [ ]:
@idom.element
async def MoreAntDesign(self):
    
    async def changed(*data):
        print(*data)

    return idom.html.div([
            antd.Button({"onClick": changed}),
            antd.Switch({"onChange": changed}),
            antd.TimePicker({"onChange": changed}),
            antd.Progress(
                {
                    "percent": "90",
                    "status": "active",
                    "strokeColor": {
                        "0%": "#108ee9",
                        "100%": "#87d068",
                    },
                }
            )
        ]
    )


print("A bunch more examples 🎉")

display(MoreAntDesign)

# Sharing Views - A Common Model State

All the examples we've shown so far display a unique view each time a client connects to the server. However it's also possible to allow clients to share server-side model state and thus (for the most part) share a common view. In order to do this we'll need to set up a `SharedClientState` server which synchronizes client model views.

## Important Note

If the server doesn't control a part of the model, then that won't be synchronized between the clients. For example the `value` of `<input/>` elements won't be synchronized unless it's explicitely declared and updated by the server:

```python
async def InputWithShareableValue(self, value=""):

    async def changed(event):
        # this update is sent to all clients
        self.update(value)

    return idom.html.input(value=value, onChange=changed)
```

By contrast this `<input/>` element's `value` would not be shared between clients:

```python
async def UnshareableInputValue(self):
    # value is not declared and updated
    return idom.html.input()
```

In [ ]:
from idom.server.sanic import SharedClientState

shared_websocket_url = example_uri_root("ws", 5678) + "/stream"
shared_webpage_url = html_link(example_uri_root("http", 5678) + "/client/index.html")

mount_shared, element = idom.hotswap(shared=True)
SharedClientState(element).daemon("localhost", 5678, access_log=False)

def shared_display(element, *args, **kwargs):
    mount_shared(element, *args, **kwargs)
    return idom.display("jupyter", shared_websocket_url)

## We Can Just Reuse the <a href="#Drag-and-Drop---Complex-Interface-Features">Drap and Drop Example</a>

In [ ]:
display = shared_display(DragDropBoxes)

In [ ]:
# display 1
display

In [ ]:
# display 2
display